In [1]:
from pathlib import Path

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from helpers.evals import evaluate_model
from helpers.loaders import prepare_data_for_pytorch
from helpers.models import FrequencyAwareNetwork
from helpers.spliters import create_frequency_based_split
from helpers.trainers import train_model

### Config


In [2]:
ANALYSIS = False
VERBOSE = True

DATASET_FILE_PATH = "dataset.csv"

GRAPH_FOLDER = "graphs"
MODELS = "models"
PREDICTIONS = "predictions"
SUBFOLDER = "baseline"

### Load


In [3]:
df = pd.read_csv(DATASET_FILE_PATH)

feature_columns = ["freq", "vb", "vc", "DEV_GEOM_L", "NUM_OF_TRANS_RF"]
label_columns = [
    "S_deemb(1,1)_real",
    "S_deemb(1,1)_imag",
    "S_deemb(1,2)_real",
    "S_deemb(1,2)_imag",
    "S_deemb(2,1)_real",
    "S_deemb(2,1)_imag",
    "S_deemb(2,2)_real",
    "S_deemb(2,2)_imag",
]

df = df.dropna(subset=feature_columns + label_columns)

X = df[feature_columns].copy()
Y = df[label_columns].copy()

### Preprocessing


In [4]:
X = pd.get_dummies(X, columns=["DEV_GEOM_L", "NUM_OF_TRANS_RF"], dtype=int)

train_mask, test_mask = create_frequency_based_split(df, test_size=0.2, random_state=42)

X_train, X_test = X[train_mask], X[test_mask]
Y_train, Y_test = Y[train_mask], Y[test_mask]

Train set: 128771 samples (82.43%)
Test set: 27443 samples (17.57%)


### Training


In [5]:
best_hparams = {
    "hidden_sizes": [384, 768, 1536],
    "dropout_rate": 0.1,
    "learning_rate": 0.002,
    "batch_size": 2048,
    "epochs": 300,
    "patience": 40,
    "activation": "gelu",
}

model_dir = Path(MODELS) / SUBFOLDER
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
freq_idx = [X_train.columns.get_loc("freq")]
other_idx = [i for i in range(X_train.shape[1]) if i not in freq_idx]

results = {}

In [6]:
# Train one model for each label pair

s_parameter_groups = [
    ("S_deemb(1,1)_real", "S_deemb(1,1)_imag"),
    ("S_deemb(1,2)_real", "S_deemb(1,2)_imag"),
    ("S_deemb(2,1)_real", "S_deemb(2,1)_imag"),
    ("S_deemb(2,2)_real", "S_deemb(2,2)_imag"),
]

for real_label, imag_label in s_parameter_groups:
    label_pair = [real_label, imag_label]
    print(f"Training model for {real_label} + {imag_label}...")
    y_train_pair = Y_train[label_pair]
    y_test_pair = Y_test[label_pair]

    X_train_tensor, Y_train_tensor, X_test_tensor, Y_test_tensor, loader, _ = (
        prepare_data_for_pytorch(
            X_train,
            y_train_pair,
            X_test,
            y_test_pair,
            batch_size=best_hparams["batch_size"],
            scale_y=False,
        )
    )

    model = FrequencyAwareNetwork(
        freq_idx,
        other_idx,
        best_hparams["hidden_sizes"],
        best_hparams["dropout_rate"],
        best_hparams["activation"],
    )
    optimizer = optim.Adam(model.parameters(), lr=best_hparams["learning_rate"])
    criterion = nn.MSELoss()

    save_path = f"{model_dir}/{real_label}_fan_model.pth"
    trained_model = train_model(
        model,
        loader,
        X_test_tensor,
        Y_test_tensor,
        criterion,
        optimizer,
        device,
        epochs=best_hparams["epochs"],
        patience=best_hparams["patience"],
        save_path=save_path,
        config=best_hparams,
        verbose=VERBOSE,
    )
    metrics = evaluate_model(
        trained_model, X_test_tensor.to(device), Y_test_tensor.to(device)
    )
    results[f"{real_label}+{imag_label}"] = metrics

Training model for S_deemb(1,1)_real + S_deemb(1,1)_imag...


Training Epochs:  14%|█▍        | 43/300 [00:11<01:07,  3.82it/s, Epoch=43, Val Loss=0.365820, Best=0.365820]                                  


Early stopping triggered.
Training model for S_deemb(1,2)_real + S_deemb(1,2)_imag...


Training Epochs:  14%|█▍        | 43/300 [00:11<01:09,  3.72it/s, Epoch=43, Val Loss=0.026853, Best=0.026852]                                  


Early stopping triggered.
Training model for S_deemb(2,1)_real + S_deemb(2,1)_imag...


Training Epochs:  53%|█████▎    | 159/300 [00:43<00:38,  3.64it/s, Epoch=159, Val Loss=4399377.500000, Best=3542721.750000]                  


Early stopping triggered.
Training model for S_deemb(2,2)_real + S_deemb(2,2)_imag...


Training Epochs: 100%|██████████| 300/300 [01:21<00:00,  3.67it/s, Epoch=300, Val Loss=0.285855, Best=0.285843]                                


### Results


In [7]:
for label_pair, metrics in results.items():
    print(f"Performance for {label_pair}:")
    print(f"  R²:   {metrics['R2'][0]:.4f}, {metrics['R2'][1]:.4f}")
    print(f"  MAE:  {metrics['MAE'][0]:.4f}, {metrics['MAE'][1]:.4f}")
    print(f"  RMSE: {metrics['RMSE'][0]:.4f}, {metrics['RMSE'][1]:.4f}")


Performance for S_deemb(1,1)_real+S_deemb(1,1)_imag:
  R²:   -0.1354, -3.8580
  MAE:  0.5693, 0.5078
  RMSE: 0.6379, 0.5698
Performance for S_deemb(1,2)_real+S_deemb(1,2)_imag:
  R²:   -1.6733, -1.5233
  MAE:  0.1502, 0.1033
  RMSE: 0.1898, 0.1329
Performance for S_deemb(2,1)_real+S_deemb(2,1)_imag:
  R²:   -20248800.0000, -12142028.0000
  MAE:  13994.5996, 6495.5195
  RMSE: 16565.6797, 7690.3262
Performance for S_deemb(2,2)_real+S_deemb(2,2)_imag:
  R²:   -1.2772, -3.3991
  MAE:  0.5989, 0.3253
  RMSE: 0.6594, 0.3700
